Here is a demo for operating the NKT Photonics SuperK COMPACT laser with SELECT tunable filter. To get started, make sure the laser is physically turned on and the interlock key is switched to the "ON" position. The laser can also be used with the Control software, but make sure this is disconnected when running the demo. There are three device classes we will be using: 

**compact** - SuperK COMPACT in charge of the overall laser power and general characteristics <br>
**select** - SuperK SELECT in charge of the two output modes (VIS/NIR and NIR/IR) <br>
**driver** - external RF driver for the SELECT, in charge of modulating frequency/wavelength for up to eight laser channels

In [ ]:
from NKTP_DLL import *
from controls import compact, select, driver

The functions used here call devices from a specific COM port. This is configured already, but can be manually changed by defining an instance of each device class. You can also check which ports are in use using the scan_ports() function. For now, let's get the laser set up. 

In [ ]:
compact.reset_interlock()
compact.emission_on()
compact.overall_power(30) # set overall power to 30%

The laser is ready, but there won't be output until we activate the RF driver as well.

In [ ]:
driver.RF_power_on()
driver.set_channel(1, 1800, 50) # set channel 1 output to 1800 nm at 50% power

Now there is emission from a single channel. Up to 8 channels can be set simultaneously. Note that the RF power is a fraction of the overall power, so the current output here is 30%*50%=15% of the total possible power. 

There are a number of other useful functions we can use. For example:

In [ ]:
compact.trig_mode()
select.crystal1_range()
compact.display_readout()

Laser mode: Internal frequency generator


When finished, set the laser back as it was by running:

In [ ]:
driver.RF_power_off()
compact.emission_off()
compact.disable_interlock()

TODO: double check the device address for RF driver

In [8]:
# TRY GETTING A RESPONSE FROM THE DIFFERENT RESULT TYPES

def wavelength_min1():
    """Crystal 1 (VIS/NIR) minimum wavelength in nm."""
    result = registerReadU32(COMport, RFdriver_devID, 0x90, -1)
    wavelength = result[1]/1000 # convert pm to nm
    print(f'Minimum wavelength for crystal 1: {int(wavelength)} nm.', RegisterStatusTypes(result))

wavelength_min1()

Minimum wavelength for crystal 1: 1680 nm. Unknown status


In [5]:
# select class

def power_monitor(self):    # CHECK
    """Readout from optional optical power monitors. Result as a percent."""
    result1 = registerReadU16(self.COMport, self.SELECT_devID, 0x10, -1)
    power1 = result1[1]/10

    result2 = registerReadU16(self.COMport, self.SELECT_devID, 0x11, -1)
    power2 = result2[1]/10

    print(f'Power monitor 1 readout: {power1} %, power monitor 2 readout: {power2} %.')

In [ ]:
def monitor_switch(self, crystal=None):  # CHECK
    """Select which power detector should be connected to the output connector. Optional input 1 or 2 to switch to the corresponding crystal."""
    if crystal==None:
        result = registerReadU8(self.COMport, self.SELECT_devID, 0x35, -1)
        result = result[1]
        if result == 0:
            print('Optical power monitor connected to crystal 1.')
        elif result == 1:
            print('Optical power monitor connected to crystal 2.')

    elif crystal == 1:
        result = registerWriteReadU8(self.COMport, self.SELECT_devID, 0x35, 0, -1)
        print('Optical power monitor connected to crystal 1.')
    elif crystal == 2:
        result = registerWriteReadU8(self.COMport, self.SELECT_devID, 0x35, 1, -1)
        print('Optical power monitor connected to crystal 2.')

    else:
        print('Invalid input. Choose 1 or 2.')